In [ ]:
# Python3 program to print the path from root 
# node to destination node for N*N-1 puzzle 
# algorithm using Branch and Bound
# The solution assumes that instance of 
# puzzle is solvable

# Importing copy for deepcopy function
import copy

# Importing the heap functions from python 
# library for Priority Queue
from heapq import heappush, heappop

# This variable can be changed to change
# the program from 8 puzzle(n=3) to 15 
# puzzle(n=4) to 24 puzzle(n=5)...
n = 3

# bottom, left, top, right
row = [ 1, 0, -1, 0 ]
col = [ 0, -1, 0, 1 ]

# A class for Priority Queue
class priorityQueue:
    
    # Constructor to initialize a
    # Priority Queue
    def __init__(self):
        self.heap = []

    # Inserts a new key 'k'
    def push(self, k):
        heappush(self.heap, k)

    # Method to remove minimum element 
    # from Priority Queue
    def pop(self):
        return heappop(self.heap)

    # Method to know if the Queue is empty
    def empty(self):
        if not self.heap:
            return True
        else:
            return False

# Node structure
class node:
    
    def __init__(self, parent, mat, empty_tile_pos,
                 cost, level):
                     
        # Stores the parent node of the 
        # current node helps in tracing 
        # path when the answer is found
        self.parent = parent

        # Stores the matrix
        self.mat = mat

        # Stores the position at which the
        # empty space tile exists in the matrix
        self.empty_tile_pos = empty_tile_pos

        # Stores the number of misplaced tiles
        self.cost = cost

        # Stores the number of moves so far
        self.level = level

    # This method is defined so that the 
    # priority queue is formed based on 
    # the cost variable of the objects
    def __lt__(self, nxt):
        return self.cost < nxt.cost

# Function to calculate the number of 
# misplaced tiles ie. number of non-blank
# tiles not in their goal position
def calculateCost(mat, final) -> int:
    
    count = 0
    for i in range(n):
        for j in range(n):
            if ((mat[i][j]) and 
                (mat[i][j] != final[i][j])):
                count += 1
                
    return count

def newNode(mat, empty_tile_pos, new_empty_tile_pos,
            level, parent, final) -> node:
                
    # Copy data from parent matrix to current matrix
    new_mat = copy.deepcopy(mat)

    # Move tile by 1 position
    x1 = empty_tile_pos[0]
    y1 = empty_tile_pos[1]
    x2 = new_empty_tile_pos[0]
    y2 = new_empty_tile_pos[1]
    new_mat[x1][y1], new_mat[x2][y2] = new_mat[x2][y2], new_mat[x1][y1]

    # Set number of misplaced tiles
    cost = calculateCost(new_mat, final)

    new_node = node(parent, new_mat, new_empty_tile_pos,
                    cost, level)
    return new_node

# Function to print the N x N matrix
def printMatrix(mat):
    
    for i in range(n):
        for j in range(n):
            print("%d " % (mat[i][j]), end = " ")
            
        print()

# Function to check if (x, y) is a valid
# matrix coordinate
def isSafe(x, y):
    
    return x >= 0 and x < n and y >= 0 and y < n

# Print path from root node to destination node
def printPath(root):
    
    if root == None:
        return
    
    printPath(root.parent)
    printMatrix(root.mat)
    print()

# Function to solve N*N - 1 puzzle algorithm
# using Branch and Bound. empty_tile_pos is
# the blank tile position in the initial state.
def solve(initial, empty_tile_pos, final):
    
    # Create a priority queue to store live
    # nodes of search tree
    pq = priorityQueue()

    # Create the root node
    cost = calculateCost(initial, final)
    root = node(None, initial, 
                empty_tile_pos, cost, 0)

    # Add root to list of live nodes
    pq.push(root)

    # Finds a live node with least cost,
    # add its children to list of live 
    # nodes and finally deletes it from 
    # the list.
    while not pq.empty():

        # Find a live node with least estimated
        # cost and delete it from the list of 
        # live nodes
        minimum = pq.pop()

        # If minimum is the answer node
        if minimum.cost == 0:
            
            # Print the path from root to
            # destination;
            printPath(minimum)
            return

        # Generate all possible children
        for i in range(4):
            new_tile_pos = [
                minimum.empty_tile_pos[0] + row[i],
                minimum.empty_tile_pos[1] + col[i], ]
                
            if isSafe(new_tile_pos[0], new_tile_pos[1]):
                
                # Create a child node
                child = newNode(minimum.mat,
                                minimum.empty_tile_pos,
                                new_tile_pos,
                                minimum.level + 1,
                                minimum, final,)

                # Add child to list of live nodes
                pq.push(child)

# Driver Code

# Initial configuration
# Value 0 is used for empty space
initial = [ [ 7, 2, 4 ], 
            [ 5, 0, 6 ], 
            [ 7, 3, 1 ] ]

# Solvable Final configuration
# Value 0 is used for empty space
final = [ [ 0, 1, 2 ], 
          [ 3, 4, 5 ], 
          [ 6, 7, 8 ] ]

# Blank tile coordinates in 
# initial configuration
empty_tile_pos = [ 1, 2 ]

# Function call to solve the puzzle
solve(initial, empty_tile_pos, final)

# This code is contributed by Kevin Joshi


In [6]:
import numpy as np
import heapq

class PuzzleState:
    def __init__(self, board, zero_pos, moves=0):
        self.board = board
        self.zero_pos = zero_pos
        self.moves = moves
        self.heuristic = self.calculate_heuristic()
        self.priority = self.moves + self.heuristic

    def calculate_heuristic(self):
        total_distance = 0
        for i in range(3):
            for j in range(3):
                value = self.board[i][j]
                if value != 0:
                    target_x = (value - 1) // 3
                    target_y = (value - 1) % 3
                    total_distance += abs(target_x - i) + abs(target_y - j)
        return total_distance

    def get_neighbors(self):
        neighbors = []
        x, y = self.zero_pos
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        
        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = self.board.copy()
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                neighbors.append(PuzzleState(new_board, (new_x, new_y), self.moves + 1))
        
        return neighbors

    def is_goal(self):
        return np.array_equal(self.board, np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))

    def __lt__(self, other):
        return self.priority < other.priority

    def print_board(self):
        for row in self.board:
            print(' '.join(str(x) if x != 0 else ' ' for x in row))
        print()

def a_star(start_board):
    zero_pos = tuple(map(tuple, np.argwhere(start_board == 0)))
    start_state = PuzzleState(start_board, zero_pos[0], 0)
    
    open_set = []
    closed_set = set()
    heapq.heappush(open_set, start_state)

    while open_set:
        current_state = heapq.heappop(open_set)

        if current_state.is_goal():
            print("Goal state reached:")
            current_state.print_board()
            return current_state.moves

        closed_set.add(tuple(map(tuple, current_state.board)))

        for neighbor in current_state.get_neighbors():
            if tuple(map(tuple, neighbor.board)) in closed_set:
                continue
            heapq.heappush(open_set, neighbor)

    return -1  # No solution found

# Example usage
initial_board =np.array([[7, 2 ,4 ], [5, 0, 6], [8, 3, 1]])
result = a_star(initial_board)
print("Number of moves to solve the puzzle:", result)


Goal state reached:
  1 2
3 4 5
6 7 8

Number of moves to solve the puzzle: 26


In [12]:
import numpy as np
import heapq

class PuzzleState:
    def __init__(self, board, zero_pos, moves=0, previous=None):
        self.board = board
        self.zero_pos = zero_pos
        self.moves = moves
        self.heuristic = self.calculate_heuristic()
        self.priority = self.moves + self.heuristic
        self.previous = previous  # Thêm tham chiếu đến trạng thái trước đó

    def calculate_heuristic(self):
        total_distance = 0
        for i in range(3):
            for j in range(3):
                value = self.board[i][j]
                if value != 0:
                    target_x = (value - 1) // 3
                    target_y = (value - 1) % 3
                    total_distance += abs(target_x - i) + abs(target_y - j)
        return total_distance

    def get_neighbors(self):
        neighbors = []
        x, y = self.zero_pos
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        
        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = self.board.copy()
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                neighbors.append(PuzzleState(new_board, (new_x, new_y), self.moves + 1, self))
        
        return neighbors

    def is_goal(self):
        return np.array_equal(self.board, np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))

    def __lt__(self, other):
        return self.priority < other.priority

    def print_board(self):
        for row in self.board:
            print(' '.join(str(x) if x != 0 else ' ' for x in row))
        print()

def a_star(start_board):
    zero_pos = tuple(map(tuple, np.argwhere(start_board == 0)))
    start_state = PuzzleState(start_board, zero_pos[0], 0)
    
    open_set = []
    closed_set = set()
    heapq.heappush(open_set, start_state)

    while open_set:
        current_state = heapq.heappop(open_set)

        if current_state.is_goal():
            # In ra các bước từ trạng thái ban đầu đến trạng thái mục tiêu
            steps = []
            while current_state:
                steps.append(current_state)
                current_state = current_state.previous
            return steps[::-1]  # Trả về danh sách các bước từ đầu đến cuối

        closed_set.add(tuple(map(tuple, current_state.board)))

        for neighbor in current_state.get_neighbors():
            if tuple(map(tuple, neighbor.board)) in closed_set:
                continue
            heapq.heappush(open_set, neighbor)

    return -1  # No solution found

# Example usage
initial_board = np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]])
solution_steps = a_star(initial_board)

if solution_steps != -1:
    print("Number of moves to solve the puzzle:", len(solution_steps) - 1)
    for step in solution_steps:
        step.print_board()
else:
    print("No solution found.")


Number of moves to solve the puzzle: 26
7 2 4
5   6
8 3 1
0
7 2 4
  5 6
8 3 1
0
  2 4
7 5 6
8 3 1
0
2   4
7 5 6
8 3 1
0
2 5 4
7   6
8 3 1
0
2 5 4
7 6  
8 3 1
0
2 5 4
7 6 1
8 3  
0
2 5 4
7 6 1
8   3
0
2 5 4
7 6 1
  8 3
0
2 5 4
  6 1
7 8 3
0
2 5 4
6   1
7 8 3
0
2 5 4
6 1  
7 8 3
0
2 5 4
6 1 3
7 8  
0
2 5 4
6 1 3
7   8
0
2 5 4
6 1 3
  7 8
0
2 5 4
  1 3
6 7 8
0
2 5 4
1   3
6 7 8
0
2 5 4
1 3  
6 7 8
0
2 5  
1 3 4
6 7 8
0
2   5
1 3 4
6 7 8
0
  2 5
1 3 4
6 7 8
0
1 2 5
  3 4
6 7 8
0
1 2 5
3   4
6 7 8
0
1 2 5
3 4  
6 7 8
0
1 2  
3 4 5
6 7 8
0
1   2
3 4 5
6 7 8
0
  1 2
3 4 5
6 7 8
0


In [11]:
import numpy as np
import heapq

class PuzzleState:
    def __init__(self, board, zero_pos, moves=0, previous=None, action=None):
        self.board = board
        self.zero_pos = zero_pos
        self.moves = moves
        self.heuristic = self.calculate_heuristic()
        self.priority = self.moves + self.heuristic
        self.previous = previous  # Tham chiếu đến trạng thái trước đó
        self.action = action  # Hành động đã thực hiện để đến trạng thái này

    def calculate_heuristic(self):
        total_distance = 0
        misplaced_tiles = 0
        for i in range(3):
            for j in range(3):
                value = self.board[i][j]
                if value != 0:
                    target_x = (value - 1) // 3
                    target_y = (value - 1) % 3
                    total_distance += abs(target_x - i) + abs(target_y - j)
                    if (i, j) != (target_x, target_y):
                        misplaced_tiles += 1
        return total_distance, misplaced_tiles

    def get_neighbors(self):
        neighbors = []
        x, y = self.zero_pos
        directions = [(1, 0, 'down'), (-1, 0, 'up'), (0, 1, 'right'), (0, -1, 'left')]
        
        for dx, dy, action in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = self.board.copy()
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                neighbors.append(PuzzleState(new_board, (new_x, new_y), self.moves + 1, self, action))
        
        return neighbors

    def is_goal(self):
        return np.array_equal(self.board, np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))

    def __lt__(self, other):
        return self.priority < other.priority

    def print_board(self):
        for row in self.board:
            print(' '.join(str(x) if x != 0 else ' ' for x in row))
        print()

def a_star(start_board):
    zero_pos = tuple(map(tuple, np.argwhere(start_board == 0)))
    start_state = PuzzleState(start_board, zero_pos[0], 0)
    
    open_set = []
    closed_set = set()
    heapq.heappush(open_set, start_state)

    while open_set:
        current_state = heapq.heappop(open_set)

        if current_state.is_goal():
            # In ra các bước di chuyển
            steps = []
            while current_state:
                steps.append(current_state)
                current_state = current_state.previous
            return steps[::-1]  # Trả về danh sách các bước từ đầu đến cuối

        closed_set.add(tuple(map(tuple, current_state.board)))

        for neighbor in current_state.get_neighbors():
            if tuple(map(tuple, neighbor.board)) in closed_set:
                continue
            heapq.heappush(open_set, neighbor)

    return -1  # No solution found

# Example usage
initial_board = np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]])
solution_steps = a_star(initial_board)

if solution_steps != -1:
    print("Number of moves to solve the puzzle:", len(solution_steps) - 1)
    for step in solution_steps:
        step.print_board()
        if step.action:
            print(f"Move: {step.action}")
        manhattan, misplaced = step.calculate_heuristic()
        print(f"Manhattan Distance: {manhattan}, Misplaced Tiles: {misplaced}")
else:
    print("No solution found.")


TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [21]:
import numpy as np
import heapq

class PuzzleState:
    def __init__(self, board, zero_pos, moves=0, previous=None):
        self.board = board
        self.zero_pos = zero_pos
        self.moves = moves  # g(x) - The number of moves so far
        self.misplaced_tiles = self.calculate_misplaced()  # h(x) - Number of misplaced tiles compared to the goal state
        self.priority = self.moves + self.misplaced_tiles  # f(x) = g(x) + h(x)
        self.previous = previous  # Reference to the previous state

    def calculate_misplaced(self):
        # Goal state
        goal_state = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])

        # Misplaced tiles - This is treated as the heuristic h(x)
        misplaced_tiles = 0
        for i in range(3):
            for j in range(3):
                # Compare each tile to the corresponding tile in the goal state
                if self.board[i][j] != 0 and self.board[i][j] != goal_state[i][j]:
                    misplaced_tiles += 1
        return misplaced_tiles

    def get_neighbors(self):
        neighbors = []
        x, y = self.zero_pos
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        
        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = self.board.copy()
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                neighbors.append(PuzzleState(new_board, (new_x, new_y), self.moves + 1, self))
        
        return neighbors

    def is_goal(self):
        return np.array_equal(self.board, np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]]))

    def __lt__(self, other):
        return self.priority < other.priority

    def print_board(self):
        for row in self.board:
            print(' '.join(str(x) if x != 0 else '0' for x in row))
        print(f"g(x) - Moves so far: {self.moves}")
        print(f"h(x) - Misplaced tiles: {self.misplaced_tiles}")
        print(f"f(x) = g(x) + h(x): {self.priority}\n")

def a_star(start_board):
    zero_pos = tuple(map(tuple, np.argwhere(start_board == 0)))
    start_state = PuzzleState(start_board, zero_pos[0], 0)
    
    open_set = []
    closed_set = set()
    heapq.heappush(open_set, start_state)

    while open_set:
        current_state = heapq.heappop(open_set)

        if current_state.is_goal():
            # Output the steps from the initial state to the goal state
            steps = []
            while current_state:
                steps.append(current_state)
                current_state = current_state.previous
            return steps[::-1]  # Return the steps in order

        closed_set.add(tuple(map(tuple, current_state.board)))

        for neighbor in current_state.get_neighbors():
            if tuple(map(tuple, neighbor.board)) in closed_set:
                continue
            heapq.heappush(open_set, neighbor)

    return -1  # No solution found

# Example usage
initial_board = np.array([[7, 2, 4], [5, 0, 6], [8, 3, 1]])
solution_steps = a_star(initial_board)

if solution_steps != -1:
    print("Number of moves to solve the puzzle:", len(solution_steps) - 1)
    for step in solution_steps:
        step.print_board()
else:
    print("No solution found.")


Number of moves to solve the puzzle: 26
7 2 4
5 0 6
8 3 1
g(x) - Moves so far: 0
h(x) - Misplaced tiles: 8
f(x) = g(x) + h(x): 8

7 2 4
0 5 6
8 3 1
g(x) - Moves so far: 1
h(x) - Misplaced tiles: 8
f(x) = g(x) + h(x): 9

0 2 4
7 5 6
8 3 1
g(x) - Moves so far: 2
h(x) - Misplaced tiles: 8
f(x) = g(x) + h(x): 10

2 0 4
7 5 6
8 3 1
g(x) - Moves so far: 3
h(x) - Misplaced tiles: 8
f(x) = g(x) + h(x): 11

2 5 4
7 0 6
8 3 1
g(x) - Moves so far: 4
h(x) - Misplaced tiles: 8
f(x) = g(x) + h(x): 12

2 5 4
7 3 6
8 0 1
g(x) - Moves so far: 5
h(x) - Misplaced tiles: 8
f(x) = g(x) + h(x): 13

2 5 4
7 3 6
0 8 1
g(x) - Moves so far: 6
h(x) - Misplaced tiles: 8
f(x) = g(x) + h(x): 14

2 5 4
0 3 6
7 8 1
g(x) - Moves so far: 7
h(x) - Misplaced tiles: 8
f(x) = g(x) + h(x): 15

2 5 4
3 0 6
7 8 1
g(x) - Moves so far: 8
h(x) - Misplaced tiles: 7
f(x) = g(x) + h(x): 15

2 5 4
3 6 0
7 8 1
g(x) - Moves so far: 9
h(x) - Misplaced tiles: 7
f(x) = g(x) + h(x): 16

2 5 0
3 6 4
7 8 1
g(x) - Moves so far: 10
h(x) - Mis